# `pathlib` Module: Taming the File System

Have you struggled with file path handling in Python? In Python 3.4 and above, the struggle is now over! You no longer need to scratch your head over code like:

```python
path.rsplit('\\', maxsplit=1)[0]
```

Or cringe at the verbosity of:

```python
os.path.isfile(os.path.join(os.path.expanduser('~'), 'realpython.txt'))
```

In this tutorial, you will see how to work with file paths—names of directories and files—in Python. You will learn new ways to read and write files, manipulate paths and the underlying file system, as well as see some examples of how to list files and iterate over them. Using the pathlib module, the two examples above can be rewritten using elegant, readable, and Pythonic code like:

```python
path.parent
(pathlib.Path.home() / 'realpython.txt').is_file()
```

## The Problem With Python File Path Handling

Working with files and interacting with the file system are important for many different reasons. The simplest cases may involve only reading or writing files, but sometimes more complex tasks are at hand. Maybe you need to list all files in a directory of a given type, find the parent directory of a given file, or create a unique file name that does not already exist.

Traditionally, Python has represented file paths using regular text strings. With support from the [`os.path`](https://docs.python.org/3/library/os.path.html) standard library, this has been adequate although a bit cumbersome (as the second example in the introduction shows). However, since paths are not strings, important functionality is spread all around the standard library, including libraries like os, glob, and shutil. The following example needs three import statements just to move all text files to an archive directory:

In [2]:
import glob
import os
import shutil

for file_name in glob.glob('*.txt'):
    print(file_name)
    new_path = os.path.join('archive', file_name)
    shutil.move(file_name, new_path)

With paths represented by strings, it is possible, but usually a bad idea, to use regular string methods. For instance, instead of joining two paths with + like regular strings, you should use `os.path.join()`, which joins paths using the correct path separator on the operating system. Recall that Windows uses `\` while Mac and Linux use `/` as a separator. This difference can lead to hard-to-spot errors, such as our first example in the introduction working for only Windows paths.

The `pathlib` module was introduced in Python 3.4 (PEP 428) to deal with these challenges. It gathers the necessary functionality in one place and makes it available through methods and properties on an easy-to-use Path object.

Early on, other packages still used strings for file paths, but as of Python 3.6, the `pathlib` module is supported throughout the standard library, partly due to the addition of a file system path protocol. If you are stuck on legacy Python, there is also a backport available for Python 2.

## Creating Paths

All you really need to know about is the pathlib.Path class. There are a few different ways of creating a path. First of all, there are classmethods like `.cwd()` (Current Working Directory) and `.home()` (your user’s home directory):

In [8]:
import pathlib

pathlib.Path.cwd()

PosixPath('/mnt/c/Users/hejaz/OneDrive/Desktop/CS-Tutorial/Python/05. Modules')

> Note: Throughout this tutorial, we will assume that `pathlib` has been imported, without spelling out `import pathlib` as above. As you will mainly be using the `Path` class, you can also do `from pathlib import Path` and write `Path` instead of `pathlib.Path`.

A path can also be explicitly created from its string representation:

In [12]:
pathlib.Path(r'C:\Users\gahjelle\realpython\file.txt')

PosixPath('C:\\Users\\gahjelle\\realpython\\file.txt')

A little tip for dealing with Windows paths: on Windows, the path separator is a backslash, `\`. However, in many contexts, backslash is also used as an escape character in order to represent non-printable characters. To avoid problems, use raw string literals to represent Windows paths. These are string literals that have an `r` prepended to them. In raw string literals the `\` represents a literal backslash: `r'C:\Users'`.

A third way to construct a path is to join the parts of the path using the special operator `/`. The forward slash operator is used independently of the actual path separator on the platform:

In [10]:
pathlib.Path.home() / 'python' / 'scripts' / 'test.py'

PosixPath('/home/ali/python/scripts/test.py')

The `/` can join several paths or a mix of paths and strings (as above) as long as there is at least one Path object. If you do not like the special `/` notation, you can do the same thing with the `.joinpath()` method:

In [11]:
pathlib.Path.home().joinpath('python', 'scripts', 'test.py')

PosixPath('/home/ali/python/scripts/test.py')

In [14]:
"prefix" / pathlib.Path("./path")

PosixPath('prefix/path')

Note that in the preceding examples, the ‍‍`pathlib.Path‍` is represented by either a `WindowsPath` or a `PosixPath`. The actual object representing the path depends on the underlying operating system. (That is, the `WindowsPath` example was run on Windows, while the `PosixPath` examples have been run on Mac or Linux.) See the section Operating System Differences for more information.

## Reading and Writing Files

Traditionally, the way to read or write a file in Python has been to use the built-in `open()` function. This is still true as the `open()` function can use `Path` objects directly. The following example finds all headers in a Markdown file and prints them:

In [32]:
path = pathlib.Path.cwd() / 'test.md'
with open(path, mode='w') as f:
    f.write("Hello World from pathlib!")

An equivalent alternative is to call `.open()` on the Path object:

In [33]:
with path.open(mode='r') as f:
    print(f.read())

Hello World from pathlib!


In fact, `Path.open()` is calling the built-in `open()` behind the scenes. Which option you use is mainly a matter of taste.

For simple reading and writing of files, there are a couple of convenience methods in the `pathlib` library:

- `.read_text()`: open the path in text mode and return the contents as a string.
- `.read_bytes()`: open the path in binary/bytes mode and return the contents as a bytestring.
- `.write_text()`: open the path and write string data to it.
- `.write_bytes()`: open the path in binary/bytes mode and write data to it.

Each of these methods handles the opening and closing of the file, making them trivial to use, for instance:

In [56]:
path = pathlib.Path.cwd() / 'test.md'
path.read_text()

'Hello World from pathlib!'

Paths can also be specified as simple file names, in which case they are interpreted relative to the current working directory. The following example is equivalent to the previous one:

In [19]:
pathlib.Path('test.md').read_text()

'Hello World from pathlib!'

The `.resolve()` method will find the full path. Below, we confirm that the current working directory is used for simple file names:

In [48]:
path = pathlib.Path('test.md')
path.resolve()

PosixPath('/mnt/c/Users/hejaz/OneDrive/Desktop/CS-Tutorial/Python/05. Modules/test.md')

In [49]:
path.resolve().parent == pathlib.Path.cwd()

True

In [50]:
path.parent == pathlib.Path.cwd()

False

Note that when paths are compared, it is their representations that are compared. In the example above, `path.parent` is not equal to `pathlib.Path.cwd()`, because path.parent is represented by `'.'` while `pathlib.Path.cwd()` is represented by `'/home/gahjelle/realpython/'`.

## Picking Out Components of a Path

The different parts of a path are conveniently available as properties. Basic examples include:

- `.name`: the file name without any directory
- `.parent`: the directory containing the file, or the parent directory if path is a directory
- `.stem`: the file name without the suffix
- `.suffix`: the file extension
- `.anchor`: the part of the path before the directories

In [51]:
path

PosixPath('test.md')

In [55]:
path.name

'test.md'

In [57]:
path.stem

'test'

In [58]:
path.suffix

'.md'

In [59]:
path.parent

PosixPath('.')

In [60]:
path.parent.parent

PosixPath('.')

In [61]:
path.anchor

''

In [73]:
path.home().anchor

'/'

Note that .parent returns a new `Path` object, whereas the other properties return strings. This means for instance that `.parent` can be chained as in the last example or even combined with `/` to create completely new paths:

In [34]:
path.parent.parent / ('new' + path.suffix)

PosixPath('new.md')

The excellent [Pathlib Cheatsheet](https://github.com/chris1610/pbpython/blob/master/extras/Pathlib-Cheatsheet.pdf) provides a visual representation of these and other properties and methods.

## Moving and Deleting Files

Through `pathlib`, you also have access to basic file system level operations like moving, updating, and even deleting files. For the most part, these methods do not give a warning or wait for confirmation before information or files are lost. Be careful when using these methods.

To move a file, use `.replace()`. Note that if the destination already exists, `.replace()` will overwrite it. Unfortunately, pathlib does not explicitly support safe moving of files. To avoid possibly overwriting the destination path, the simplest is to test whether the destination exists before replacing:

```python
if not destination.exists():
    source.replace(destination)
```    

When you are renaming files, useful methods might be `.with_name()` and `.with_suffix()`. They both return the original path but with the name or the suffix replaced, respectively.

For instance:

In [58]:
path.with_suffix('.py')

PosixPath('/mnt/c/Users/hejaz/OneDrive/Desktop/CS-Tutorial/Python/05. Modules/test.py')

In [59]:
path.replace(path.with_suffix('.py'))

Directories and files can be deleted using `.rmdir()` and `.unlink()` respectively. (Again, be careful!)

In [86]:
Path.cwd() / "main.py"

PosixPath('/mnt/c/Users/hejaz/OneDrive/Desktop/CS-Tutorial/Python/05. Modules/main.py')

## Examples

In this section, you will see some examples of how to use `pathlib` to deal with simple challenges.

### Counting Files

There are a few different ways to list many files. The simplest is the `.iterdir()` method, which iterates over all files in the given directory. The following example combines `.iterdir()` with the `collections.Counter` class to count how many files there are of each filetype in the current directory:

In [64]:
import collections

collections.Counter(p.suffix for p in pathlib.Path.cwd().iterdir())

Counter({'': 1, '.ipynb': 4, '.yaml': 1, '.py': 1})

## Find the Last Modified File

To find the file in a directory that was last modified, you can use the `.stat()` method to get information about the underlying files. For instance, `.stat().st_mtime` gives the time of last modification of a file:

In [116]:
from datetime import datetime

In [126]:
path.resolve().stat()

os.stat_result(st_mode=33188, st_ino=3377699720870481, st_dev=14, st_nlink=1, st_uid=1000, st_gid=1000, st_size=25, st_atime=1621214763, st_mtime=1621214762, st_ctime=1621214766)

In [119]:
[(f.stat().st_mtime, f) for f in path.parent.iterdir()]

[(1621206494.6305218, PosixPath('.ipynb_checkpoints')),
 (1621213214.6805825, PosixPath('14.1 json.ipynb')),
 (1621213214.6876798, PosixPath('14.2 pickle.ipynb')),
 (1621212512.615489, PosixPath('14.3 yaml.ipynb')),
 (1621206166.2721207, PosixPath('collections.ipynb')),
 (1621210977.7855675, PosixPath('input.txt')),
 (1621210958.4577336, PosixPath('learn_yaml.yaml')),
 (1621213340.4888136, PosixPath('main.py')),
 (1621214693.9817107, PosixPath('pathlib.ipynb')),
 (1621214762.334, PosixPath('test.md')),
 (1621208346.1684408, PosixPath('time.ipynb'))]

In [120]:
time, file_path = max((f.stat().st_mtime, f) for f in path.parent.iterdir())

You can even get the contents of the file that was last modified with a similar expression:

In [121]:
print(datetime.fromtimestamp(time), file_path)

2021-05-16 21:26:02.334000 test.md


The timestamp returned from the different `.stat().st_ properties` represents seconds since **January 1st, 1970**. In addition to `datetime.fromtimestamp`, `time.localtime` or `time.ctime` may be used to convert the timestamp to something more usable.

### Create a Unique File Name

The last example will show how to construct a unique numbered file name based on a template. First, specify a pattern for the file name, with room for a counter. Then, check the existence of the file path created by joining a directory and the file name (with a value for the counter). If it already exists, increase the counter and try again:

In [129]:
def unique_path(directory, name_pattern):
    counter = 0
    while True:
        counter += 1
        path = directory / name_pattern.format(counter)
        if not path.exists():
            return path

path = unique_path(pathlib.Path.cwd(), 'test{:03d}.txt')

In [130]:
path

PosixPath('/mnt/c/Users/hejaz/OneDrive/Desktop/CS-Tutorial/Python/05. Modules/test001.txt')

If the directory already contains the files `test001.txt` and `test002.txt`, the above code will set path to `test003.txt`.

## Conclusion

Since Python 3.4, pathlib has been available in the standard library. With pathlib, file paths can be represented by proper Path objects instead of plain strings as before. These objects make code dealing with file paths:

- Easier to read, especially because `/` is used to join paths together
- More powerful, with most necessary methods and properties available directly on the object
- More consistent across operating systems, as peculiarities of the different systems are hidden by the `Path` object

In this tutorial, you have seen how to create Path objects, read and write files, manipulate paths and the underlying file system, as well as some examples of how to iterate over many file paths.